<a href="https://colab.research.google.com/github/shreyashkar-ml/Pytorch-learning/blob/main/Automated_HyperParameter_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import inspect

In [3]:
class Config:
  def __init__(self, **kwargs):
    frame = inspect.currentframe()
    local_vars = inspect.getargvalues(frame).locals
    for key, value in local_vars['kwargs'].items():
      setattr(self, key, value)

In [4]:
# Example usage
app_config = Config(db_name="my_database", max_connections=100, debug = True)

print(app_config.db_name)
print(app_config.max_connections)
print(app_config.debug)

my_database
100
True


#### Example in Deep Learning

In [5]:
class HyperParameterConfig:
  def __init__(self, **kwargs):
    frame = inspect.currentframe()
    local_vars = inspect.getargvalues(frame).locals
    for key, value in local_vars['kwargs'].items():
      setattr(self, key, value)

In [6]:
hyperparams = HyperParameterConfig(
    learning_rate = 0.001,
    batch_size = 64,
    epochs = 10,
    optimizer = "adam",
    dropout_rate = 0.2,
    hidden_units = [128,64],
)

# Access hyperparameters
print(hyperparams.learning_rate)
print(hyperparams.batch_size)
print(hyperparams.epochs)
print(hyperparams.optimizer)
print(hyperparams.dropout_rate)
print(hyperparams.hidden_units)

0.001
64
10
adam
0.2
[128, 64]


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import inspect

In [10]:
class SimpleNN(nn.Module):
  def __init__(self, input_size, hidden_units, dropout_rate, n_classes = 10):
    super(SimpleNN, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_units[0])
    self.fc2 = nn.Linear(hidden_units[0], hidden_units[1])
    self.dropout = nn.Dropout(dropout_rate)
    self.fc3 = nn.Linear(hidden_units[1], n_classes)

  def forward(self, x):
    x = x.view(-1, 28*28)         # Flatten the input --> 28*28 --> 784
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.relu(self.fc2(x))
    return self.fc3(x)

In [23]:
hyperparams = HyperParameterConfig(
    learning_rate = 0.001,
    batch_size = 64,
    epochs = 5,
    optimizer = "adam",
    dropout_rate = 0.2,
    hidden_units = [128,64],
)

# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform = transform)
train_loader = DataLoader(train_dataset, batch_size = hyperparams.batch_size, shuffle=True)

# Instantiate Model and Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 28*28

model = SimpleNN(input_size, hyperparams.hidden_units, hyperparams.dropout_rate).to(device)

optimizer = optim.Adam(model.parameters(), lr=hyperparams.learning_rate) if hyperparams.optimizer == "adam" else optim.sgd(model.parameters(), lr=hyperparams.learning_rate)
criterion = nn.CrossEntropyLoss()

In [26]:
def train_model(model, optimizer, criterion, epochs, train_loader):
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()     # Zero the parameter gradients
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()           # Backpropagate the loss

      optimizer.step()          # Update weights

      running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [27]:
train_model(model, optimizer, criterion, hyperparams.epochs, train_loader)

Epoch 1/5, Loss: 0.13177062278248067
Epoch 2/5, Loss: 0.12357729150137223
Epoch 3/5, Loss: 0.11617696313842782
Epoch 4/5, Loss: 0.11015708800822274
Epoch 5/5, Loss: 0.1065674691047988
